<strong>data topic : Bank marketing campaigns dataset</strong>

Portugal bank marketing campaigns results.
If after all marking afforts client had agreed to place deposit - target variable marked 'yes', otherwise 'no'

Number of Instances: 41188 for bank-additional-full.csv


dataset url = https://www.kaggle.com/datasets/volodymyrgavrysh/bank-marketing-campaigns-dataset

#### 앙상블 기법 중에서 bagging 기법 중 대표적인 모델인 <strong>'랜덤 포레스트'</strong>을 이용해서 진행할 것입니다.

##### 앙상블의 기본적인 개념은 모델을 많이, 다양하게 만들어서 합치는 것이며, 이렇게 하기 위해서는 2가지 과정이 필요하다.

- 1. 각 모델의 다양성을 확보한다.
  - random sampling 진행시켜서 여러 데이터셋 생성(원본의 값이 중복되거나, 없거나 할 수 있다.)
  - 각 데이터셋 모델을 만들기
  - 모델 별로 학습하는 데이터셋의 다양성으로 인해 모델 다양성 확보하기
- 2. 각 결과물들 어떻게 결합할 것인가? 
  - 단순 평균을 통해 예측 평균값 구하기

In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/datagit/data/bank-additional-full.csv', sep=';')

In [5]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [6]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

one-hot encoding을 잔행해서 학습에 이용할 수 있게 숫자 형태로 데이터를 바꿔줍니다.

In [12]:
df = pd.get_dummies(df, columns = ['job', 'marital', 'education','default', 'housing', 'loan', 'contact', 'month', 'day_of_week','poutcome'])
df

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,0,0,0,1,0
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,0,0,0,1,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,0,0,0,1,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,0,0,0,1,0


In [13]:
df['id'] = range(len(df))
len(df)

41188

train, test을 만들어주기 위해 dataset을 나눠준다.

In [15]:
train = df.sample(30000, replace=False,random_state=2020).reset_index().drop(['index'], axis=1)
test = df.loc[- df['id'].isin(train['id'])].reset_index().drop(['index'],axis=1)

### randomforest 진행과 의사결정나무 한번 사용한 결과를 비교해보자

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [31]:
df = DecisionTreeClassifier(min_samples_split=10)

In [18]:
train.columns

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'mon

In [23]:
# train['y']은 예측해야 할 값이니까 제외시켜준다.
input_var = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'id']

<strong>의사결정나무를 진행해보자</strong>

In [32]:
df.fit(train[input_var], train['y'])

test['pred'] = df.predict(test[input_var])

In [34]:
test['pred']

0         no
1         no
2         no
3         no
4         no
        ... 
11183     no
11184     no
11185     no
11186     no
11187    yes
Name: pred, Length: 11188, dtype: object

In [33]:
(test['pred'] == test['y']).mean()

0.8959599570968895

<strong>랜덤포레스트 진행해보자</strong>

In [30]:
# n_estimators : 몇 개의 의사결정나무를 만들 것인지
# min_samples_split : 각 의사결정나무에서 각 구간의 최소 샘플 수, 이보다 작으면 더 이상 분리되지않는다.

rf = RandomForestClassifier(n_estimators = 500, min_samples_split=10)

In [25]:
rf.fit(train[input_var],train['y'])

RandomForestClassifier(min_samples_split=10, n_estimators=500)

In [26]:
predictions = rf.predict(test[input_var])

In [27]:
test['pred'] = predictions

In [28]:
(test['pred'] == test['y']).mean()

0.9138362531283518

#### 앙상블 기법 중에서 부스팅 기법 중 대표적인 모델인 <strong>'XGBoost'</strong>을 이용해서 진행할 것입니다.


- 1. 각 모델의 다양성을 확보한다.
  - 이전 모델에서 오분류한 객체에 가중치를 높여서 새로운 데이터로 모델 학습
  - 각 데이터셋 모델을 만들기
  - 모델 별로 학습하는 데이터셋의 다양성으로 인해 모델 다양성 확보하기
- 2. 각 결과물들 어떻게 결합할 것인가? 
  - 각 모델로부터 나온 예측치 가중평균한다.

##### learning_rate : 얼마나 빠르게 학습할 것인지(가중치를 얼마나 극단적으로 업데이트 시킬것인지)

만들 모델의 개수는 가중치를 몇 번 업데이트하는지에 대한 내용이다.

- n_estimators이 너무 높으면 노이즈에 민감한 오버피팅
- n_estimators이 너무 작으면 언더피팅의 우려가 있다.

   => 적절한 n_estimators 찾기(Knn에서 적절한 k 찾는 방법이랑 비슷하다)

In [35]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
from xgboost import XGBClassifier

In [46]:
xgb = XGBClassifier(n_estimators=300, learning_rate=0.1)

In [47]:
xgb.fit(train[input_var], train['y'])

XGBClassifier(n_estimators=300)

In [48]:
predictions = xgb.predict(test[input_var])

In [49]:
test['pred'] = predictions

In [50]:
(test['pred'] == test['y']).mean()

0.9149982123703968

최적의 n_estimators 찾기

In [ ]:
for n in [100,200,300,400,500,600,700,800,900]:
  
    xgb = XGBClassifier(n_estimators=n, learning_rate=0.1)
    xgb.fit(train[input_var], train['y'])
    predictions = xgb.predict(test[input_var])
    test['pred'] = predictions
    print((test['pred'] == test['y']).mean())